In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# Load the dataset

In [2]:
df = pd.read_csv("car_prediction_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'car_prediction_data.csv'

In [ ]:
df.head()

In [ ]:
print("Data type : ", type(df))
print("Data dims : ", df.shape)

In [ ]:
print(df.dtypes)

In [ ]:
df.isnull().sum()

In [ ]:
print(df['Fuel_Type'].unique())
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
df.describe()

In [ ]:
df = df.drop(['Car_Name'], axis=1)
df.head()

In [ ]:
df['No_of_years'] = 2023 - df['Year']
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop(['Year'], axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
Selling_Price = pd.DataFrame(df['Selling_Price'])
Present_Price  = pd.DataFrame(df['Present_Price'])
Kms_Driven = pd.DataFrame(df['Kms_Driven'])


In [ ]:
f, axes = plt.subplots(3, 3, figsize= (12, 20))


sb.boxplot(data = Present_Price, orient = "h", ax = axes[0,0])
sb.histplot(data = Present_Price, ax = axes[1,0], kde = True)
sb.violinplot(data = Present_Price, orient = "h", ax = axes[2,0])

sb.boxplot(data = Kms_Driven, orient = "h", ax = axes[0,1])
sb.histplot(data = Kms_Driven, ax = axes[1,1], kde = True)
sb.violinplot(data = Kms_Driven, orient = "h", ax = axes[2,1])


sb.boxplot(data = No_of_years, orient = "h", ax = axes[0,1])
sb.histplot(data = No_of_years, ax = axes[1,2], kde = True)
sb.violinplot(data = No_of_years, orient = "h", ax = axes[2,2])


f = plt.figure()

In [ ]:
extracted_numeric_data = df[['Present_Price','Kms_Driven','No_of_years']]
extracted_numeric_data.skew(axis = 0, skipna = True)

In [ ]:
q1 = extracted_numeric_data.quantile(0.25)
q3 = extracted_numeric_data.quantile(0.75)
iqr = q3-q1
outliers = ((extracted_numeric_data < q1-1.5*iqr) | (extracted_numeric_data > q3+1.5*iqr)).sum()
outliers

In [ ]:
df = pd.get_dummies(df,drop_first=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# Correlation between Response and the Predictors
corr_res_pred = pd.concat([df['Selling_Price'], df.drop(['Selling_Price'], axis=1)], axis = 1).reindex(df['Selling_Price'].index)
# jointPlot = pd.concat([data_encoded["time_spent"], data_encoded[['income_time_spent_interaction','age_time_spent_interaction','age_income_interaction']]], axis=1)

# jointPlot.corr()
f = plt.figure(figsize=(12, 8))
sb.heatmap(corr_res_pred.corr(), vmin = -1, vmax = 1, annot = True, fmt = ".2f")

# Linear Regression Attempt 1

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
#Extract Response and Predictors
y = pd.DataFrame(df['Selling_Price'])
X = pd.DataFrame(df.drop(columns=['Selling_Price']))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=32)
# Set random state to 
# ensure the data will be split into the same training and testing sets every time the code is executed unless the test_size is changed

In [ ]:
# Linear Regression using Train Data
linreg = LinearRegression()         # create the linear regression object
linreg.fit(X_train, y_train)        # train the linear regression model

# Coefficients of the Linear Regression line
print('Intercept of Regression \t: b = ', linreg.intercept_)
print('Coefficients of Regression \t: a = ', linreg.coef_)
print()

# Print the Coefficients against Predictors
print(pd.DataFrame(list(zip(X_train, linreg.coef_[0])), columns = ["Predictors", "Coefficients"]))
print()

# Predict Response corresponding to Predictors
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

In [ ]:
# Plot the Predictions vs the True values
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train, y_train_pred, color = "blue")
axes[0].plot(y_train, y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test, y_test_pred, color = "green")
axes[1].plot(y_test, y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train, y_train_pred))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print()

# Linear Regression Attempt 2

Attempt to remove outliers from dataset

In [ ]:
# Identifying outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
outliers_mask = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)

In [ ]:
# Removing outliers
df = df[~outliers_mask]

# Random Forest Regressor Attempt 1

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=32, oob_score=True,max_depth = 3)


model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
# Plot the Predictions vs the True values
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train, y_train_pred, color = "blue")
axes[0].plot(y_train, y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test, y_test_pred, color = "green")
axes[1].plot(y_test, y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", model.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train, y_train_pred))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", model.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print()